In [13]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import _canoeValues as cv

#-------------------Inputs-------------------------------------------------------------
input_files = ["Shear_and_Moment_6 Paddlers.csv", "Shear_and_Moment_4 Paddlers.csv", "Shear_and_Moment_3 Paddlers.csv", "Shear_and_Moment_2 Paddlers.csv", "Shear_and_Moment_Display_Stand.csv"]
inner_file = "Inner Hull.csv"
outer_file = "Outer Hull.csv"
compressive_strength = cv.compressive_strength # MPa
#--------------------------------------------------------------------------------------

script_dir = os.path.abspath('')

file_path = os.path.join(script_dir, inner_file) # file path for the text file input
in_coor = pd.read_csv(file_path, header=None)

file_path = os.path.join(script_dir, outer_file) # file path for the text file input
out_coor = pd.read_csv(file_path, header=None)

#x_coor = coordinates[0].iloc[1:] # get rid of [0,0,0]
x_in = in_coor[0]
y_in = in_coor[1]
z_in = in_coor[2]

x_out = out_coor[0]
y_out = out_coor[1]
z_out = out_coor[2]

station_no = x_out.nunique() # number of stations
station_out = x_out.unique() # location of each station
station_in = x_in.unique()


In [14]:
print("Positive Moments:\n")
# Loop through each script
for input_file in input_files:
    print("\033[4m" + input_file + ":\033[0m")
    file_path = os.path.join(script_dir, input_file)
    info = pd.read_csv(file_path, header=None)

    # Positive Bending Moment
    value = float(info[1][3])
    width = float(info[3][3])
    depth = float(info[4][3])
    target_station = float(info[2][3])

    # Extract Station Information
    Y1_in = []; Z1_in = []
    Y2_in = []; Z2_in = []
    Y1_out = []; Z1_out = []
    Y2_out = []; Z2_out = []

    for k in range(1,station_no-2,1):
        if k == target_station:
            X1_in = station_in[k]
            X2_in = station_in[k+1]
            X1_out = station_out[k]
            X2_out = station_out[k+1]
            
            for i in range(0,len(x_out),1):
                if x_out[i] == X1_out:
                    Y1_out.append(y_out[i])
                    Z1_out.append(z_out[i])

                elif x_out[i] == X2_out:
                    Y2_out.append(y_out[i])
                    Z2_out.append(z_out[i])
                
                if x_in[i] == X1_in:
                    Y1_in.append(y_in[i])
                    Z1_in.append(z_in[i])

                elif x_in[i] == X2_in:
                    Y2_in.append(y_in[i])
                    Z2_in.append(z_in[i])

    # Find y_bar
    volume = 0
    product = 0 # y_bar*Volume
    for i in range(0,len(Y1_out)-1,1):
        z_i = ((Z1_out[i]+Z1_out[i+1]+Z1_in[i]+Z1_in[i+1])/4 + (Z2_out[i]+Z2_out[i+1]+Z2_in[i]+Z2_in[i+1])/4) /2
        x = (abs(X1_in-X2_in) + abs(X1_out-X2_out))/2
        y = (abs(Y1_in[i] - Y1_out[i]) + abs(Y1_in[i+1]-Y1_out[i+1]) + abs(Y2_in[i] - Y2_out[i]) + abs(Y2_in[i+1]-Y2_out[i+1]))/4
        z = (abs(Z1_out[i+1]-Z1_out[i]) + abs(Z1_in[i+1]-Z1_in[i]) + abs(Z2_out[i+1]-Z2_out[i]) + abs(Z2_in[i+1]-Z2_in[i]))/4

        volume_i = x * y * z
        volume += volume_i
        product += volume_i * z_i
        
    y_bar_top = abs(product/volume)
    y_bar_bottom = depth - y_bar_top

    # Find I
    I = 0
    for i in range(0,len(Y1_out)-1,1):
        z_i = ((Z1_out[i]+Z1_out[i+1]+Z1_in[i]+Z1_in[i+1])/4 + (Z2_out[i]+Z2_out[i+1]+Z2_in[i]+Z2_in[i+1])/4) /2
        y = (abs(Y1_in[i] - Y1_out[i]) + abs(Y1_in[i+1]-Y1_out[i+1]) + abs(Y2_in[i] - Y2_out[i]) + abs(Y2_in[i+1]-Y2_out[i+1]))/4
        z = (abs(Z1_out[i+1]-Z1_out[i]) + abs(Z1_in[i+1]-Z1_in[i]) + abs(Z2_out[i+1]-Z2_out[i]) + abs(Z2_in[i+1]-Z2_in[i]))/4
        I_i = (1/12*y*z**3 + y*z*(abs(z_i) - y_bar_top)**2)/(10**(12)) # [m^4]
        I = I + I_i

    stress_top = value*(y_bar_top/1000)/I
    stress_bottom = value*(y_bar_bottom/1000)/I

    resistance_top = compressive_strength * I / (y_bar_top/1000) * 10**6  # Nm
    resistance_bottom = compressive_strength * I / (y_bar_bottom/1000) * 10**6  # Nm

    print('Compressive Flexural Stress = ', stress_top/10**6, ' MPa')
    print('Tensile Flexural Stress = ', stress_bottom/10**6, ' MPa')
    print("Applied Positive Moment: " + str(value) + ' Nm')
    print("Resistance Top: " + str(resistance_top) + " Nm")
    print("Resistance Bottom: " + str(resistance_bottom) + " Nm")
    
    print("Depth: " + str(depth))
    print("I: " + str(I))
    print("Ybar Top: " + str(y_bar_top))
    print("Ybar Bottom: " + str(y_bar_bottom))
    print("\n")

Positive Moments:

Shear_and_Moment_6 Paddlers.csv:
Compressive Flexural Stress =  0.8771970996337307  MPa
Tensile Flexural Stress =  1.0478998168205158  MPa
Applied Positive Moment: 635.6105314577962 Nm
Resistance Top: 12303.582431656243 Nm
Resistance Bottom: 10299.33076703834 Nm
Depth: 342.5
I: 0.00011308345318284611
Ybar Top: 156.06487343916186
Ybar Bottom: 186.43512656083814


Shear_and_Moment_4 Paddlers.csv:
Compressive Flexural Stress =  0.6893584496246877  MPa
Tensile Flexural Stress =  0.8235077400150768  MPa
Applied Positive Moment: 454.94184066759146 Nm
Resistance Top: 11205.944394736049 Nm
Resistance Bottom: 9380.497691975885 Nm
Depth: 342.5
I: 0.0001029949518970693
Ybar Top: 156.0648725005056
Ybar Bottom: 186.4351274994944


Shear_and_Moment_3 Paddlers.csv:
Compressive Flexural Stress =  0.5632376609439006  MPa
Tensile Flexural Stress =  0.672843820958191  MPa
Applied Positive Moment: 371.70847523901915 Nm
Resistance Top: 11205.944394736049 Nm
Resistance Bottom: 9380.497691

In [15]:
print("Negative Moments:\n")
# Loop through each script
for input_file in input_files:
    print("\033[4m" + input_file + ":\033[0m")
    file_path = os.path.join(script_dir, input_file)
    info = pd.read_csv(file_path, header=None)

    value = float(info[1][4])
    width = float(info[3][4])
    depth = float(info[4][4])
    target_station = float(info[2][4])

    # Extract Station Information
    Y1_in = []; Z1_in = []
    Y2_in = []; Z2_in = []
    Y1_out = []; Z1_out = []
    Y2_out = []; Z2_out = []

    for k in range(1,station_no-2,1):
        if k == target_station:
            X1_in = station_in[k]
            X2_in = station_in[k+1]
            X1_out = station_out[k]
            X2_out = station_out[k+1]
            
            for i in range(0,len(x_out),1):
                if x_out[i] == X1_out:
                    Y1_out.append(y_out[i])
                    Z1_out.append(z_out[i])

                elif x_out[i] == X2_out:
                    Y2_out.append(y_out[i])
                    Z2_out.append(z_out[i])
                
                if x_in[i] == X1_in:
                    Y1_in.append(y_in[i])
                    Z1_in.append(z_in[i])

                elif x_in[i] == X2_in:
                    Y2_in.append(y_in[i])
                    Z2_in.append(z_in[i])

    # Find y_bar
    volume = 0
    product = 0 # y_bar*Volume

    depth = 0

    for i in range(0,len(Y1_out)-1,1):
        z_i = ((Z1_out[i]+Z1_out[i+1]+Z1_in[i]+Z1_in[i+1])/4 + (Z2_out[i]+Z2_out[i+1]+Z2_in[i]+Z2_in[i+1])/4) /2
        x = (abs(X1_in-X2_in) + abs(X1_out-X2_out))/2
        y = (abs(Y1_in[i] - Y1_out[i]) + abs(Y1_in[i+1]-Y1_out[i+1]) + abs(Y2_in[i] - Y2_out[i]) + abs(Y2_in[i+1]-Y2_out[i+1]))/4
        z = (abs(Z1_out[i+1]-Z1_out[i]) + abs(Z1_in[i+1]-Z1_in[i]) + abs(Z2_out[i+1]-Z2_out[i]) + abs(Z2_in[i+1]-Z2_in[i]))/4

        volume_i = y * z # * x
        volume += volume_i
        product += volume_i * z_i

        depth = max(depth, abs(min(Z1_out)), abs(min(Z2_out)))
        
    y_bar_top = abs(product/volume)
    y_bar_bottom = depth - y_bar_top

    # Find I
    I = 0
    for i in range(0,len(Y1_out)-1,1):
        z_i = ((Z1_out[i]+Z1_out[i+1]+Z1_in[i]+Z1_in[i+1])/4 + (Z2_out[i]+Z2_out[i+1]+Z2_in[i]+Z2_in[i+1])/4) /2
        y = (abs(Y1_in[i] - Y1_out[i]) + abs(Y1_in[i+1]-Y1_out[i+1]) + abs(Y2_in[i] - Y2_out[i]) + abs(Y2_in[i+1]-Y2_out[i+1]))/4
        z = (abs(Z1_out[i+1]-Z1_out[i]) + abs(Z1_in[i+1]-Z1_in[i]) + abs(Z2_out[i+1]-Z2_out[i]) + abs(Z2_in[i+1]-Z2_in[i]))/4
        I_i = (1/12*y*z**3 + y*z*(abs(z_i) - y_bar_top)**2)/10**(12) # [m^4]
        I = I + I_i

    stress_top = value*(y_bar_top/1000)/I
    stress_bottom = value*(y_bar_bottom/1000)/I

    resistance_top = compressive_strength * I / (y_bar_top/1000) * 10**6  # Nm
    resistance_bottom = compressive_strength * I / (y_bar_bottom/1000) * 10**6  # Nm

    print('Tensile Flexural Stress = ', stress_top/10**6, ' MPa')
    print('Compressive Flexural Stress = ', stress_bottom/10**6, ' MPa')
    print("Applied Negative Moment: " + str(value) + ' Nm')
    print("Stress Top: " + str(stress_top) + " MPa")
    print("Stress Bottom: " + str(stress_bottom) + " MPa")
    print("Resistance Top: " + str(resistance_top) + " Nm")
    print("Resistance Bottom: " + str(resistance_bottom) + " Nm")
    print("\n")


Negative Moments:

Shear_and_Moment_6 Paddlers.csv:
Tensile Flexural Stress =  0.0  MPa
Compressive Flexural Stress =  0.0  MPa
Applied Negative Moment: 0.0 Nm
Stress Top: 0.0 MPa
Stress Bottom: 0.0 MPa
Resistance Top: 963.5236256509168 Nm
Resistance Bottom: 563.8313058774876 Nm


Shear_and_Moment_4 Paddlers.csv:
Tensile Flexural Stress =  -0.19219314202634732  MPa
Compressive Flexural Stress =  -0.32843623810896605  MPa
Applied Negative Moment: -10.905926562453933 Nm
Stress Top: -192193.1420263473 MPa
Stress Bottom: -328436.23810896603 MPa
Resistance Top: 963.5236256509169 Nm
Resistance Bottom: 563.8313058774877 Nm


Shear_and_Moment_3 Paddlers.csv:
Tensile Flexural Stress =  -11.558829972672331  MPa
Compressive Flexural Stress =  -19.752726830623498  MPa
Applied Negative Moment: -655.9013995024578 Nm
Stress Top: -11558829.972672332 MPa
Stress Bottom: -19752726.830623496 MPa
Resistance Top: 963.5236256509169 Nm
Resistance Bottom: 563.8313058774877 Nm


Shear_and_Moment_2 Paddlers.csv: